### Feature Engineering  
In diesem Notebook wird die **raw** CSV "Datengrundlage_Projekt" um weitere Spalten ergänzt, die möglicherweise weitere Predictors sein könnten.

#### Imports

In [1]:
import pandas as pd

#### Daten laden

In [2]:
path_data = 'c:/Users/miche/HdM/Data Analytics with Statistics/Git/project/data/raw/'
file_data = 'Datengrundlage_Projekt.csv'

data = pd.read_csv(path_data + file_data, sep=',', encoding='utf-8')

#### Überblick über raw-data

In [3]:
data.head()

,MONATE_SEIT_EINFUEHRUNG_PROGRAMM_KOHORTE,MONAT,KOHORTE,ERSTER_MONAT_KOHORTE_FG,MONATE_SEIT_EXISTENZ_KOHORTE,KOHORTENGROESSE_INDEXIERT,KOHORTENGROESSE_KUMULIERT_INDEXIERT,IDENTIFIZIERTE_KUNDEN_INDEXIERT,RETENTIONRATE,UMSATZ_INDEXIERT,...,UMSATZANTEIL_KOHORTE,ANZ_BONS_INDEXIERT,ABVERKAUF_INDEXIERT,RABATT_INDEXIERT,FREQUENZ_INDEXIERT,DURCHSCHNITTSBON_INDEXIERT,STUECKBON_INDEXIERT,UMSATZ_JE_KUNDE_INDEXIERT,ABVERKAUF_JE_KUNDE_INDEXIERT,RABATT_JE_KUNDE_INDEXIERT
0,-2,201408,201408,1,0,0.41,0.08,0.41,100.0000,0.16,...,0.104058,0.54,0.20,2.54,131.29,29.28,37.51,38.45,49.24,615.67
1,-2,201409,201408,0,1,0.41,0.08,0.40,95.8466,1.14,...,0.007668,2.54,1.41,7.14,641.94,44.77,55.55,287.41,356.62,1805.14
2,-2,201410,201408,0,2,0.41,0.08,0.39,94.2492,1.58,...,0.000270,3.38,1.85,9.28,868.77,46.60,54.67,404.81,474.95,2385.68
3,-2,201411,201408,0,3,0.41,0.08,0.38,91.3738,1.33,...,0.000226,2.92,1.44,3.22,774.33,45.65,49.19,353.50,380.91,852.38
4,-2,201412,201408,0,4,0.41,0.08,0.38,91.3738,1.76,...,0.000239,2.93,1.71,7.15,776.19,59.98,58.46,465.54,453.73,1894.07


In [4]:
data.columns

Index(['MONATE_SEIT_EINFUEHRUNG_PROGRAMM_KOHORTE', 'MONAT', 'KOHORTE',
       'ERSTER_MONAT_KOHORTE_FG', 'MONATE_SEIT_EXISTENZ_KOHORTE',
       'KOHORTENGROESSE_INDEXIERT', 'KOHORTENGROESSE_KUMULIERT_INDEXIERT',
       'IDENTIFIZIERTE_KUNDEN_INDEXIERT', 'RETENTIONRATE', 'UMSATZ_INDEXIERT',
       'MONATLICHER_UMSATZ_INDEXIERT', 'UMSATZANTEIL_KOHORTE',
       'ANZ_BONS_INDEXIERT', 'ABVERKAUF_INDEXIERT', 'RABATT_INDEXIERT',
       'FREQUENZ_INDEXIERT', 'DURCHSCHNITTSBON_INDEXIERT',
       'STUECKBON_INDEXIERT', 'UMSATZ_JE_KUNDE_INDEXIERT',
       'ABVERKAUF_JE_KUNDE_INDEXIERT', 'RABATT_JE_KUNDE_INDEXIERT'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703 entries, 0 to 702
Data columns (total 21 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   MONATE_SEIT_EINFUEHRUNG_PROGRAMM_KOHORTE  703 non-null    int64  
 1   MONAT                                     703 non-null    int64  
 2   KOHORTE                                   703 non-null    int64  
 3   ERSTER_MONAT_KOHORTE_FG                   703 non-null    int64  
 4   MONATE_SEIT_EXISTENZ_KOHORTE              703 non-null    int64  
 5   KOHORTENGROESSE_INDEXIERT                 703 non-null    float64
 6   KOHORTENGROESSE_KUMULIERT_INDEXIERT       703 non-null    float64
 7   IDENTIFIZIERTE_KUNDEN_INDEXIERT           703 non-null    float64
 8   RETENTIONRATE                             703 non-null    float64
 9   UMSATZ_INDEXIERT                          703 non-null    float64
 10  MONATLICHER_UMSATZ_INDEXIERT          

#### Feature Engineering  
Nach Betrachtung der vorhandenen Spalten werden weitere Zeitvariablen als sinnvoll erachtet. Aktuell werden der Monat sowie die Kohort im Format JJJJMM angegeben.  
Damit das Modell Saisonalitäten zur Jahreszeit, zum Monat sowie einen allgemeinen Trend bei zunehmender Jahreszahl erkennen kann, werden die Spalten `Monat` und `Kohorte` in weitere Zeitvariblen zerlegt.

Um die Werte in den Spalten zerlegen zu können, werden zunächst zwei Funktionen definiert.  
- `left()` gibt den linken Part eines Werts aus. In der Funktion kann definiert werden, wie viele Zeichen des linken Parts ausgegeben werden.  
- `right()` gibt den rechten Part eines Werts aus. In der Funktion kann definiert werden, wie viele Zeichen des rechten Parts ausgegeben werden.  

In [15]:
def left(val, n = 4):
    '''
    Extrahiert die ersten n-Zeichen eines Strings.

    Args:
    val (str): Der Eingabestring.
    n (int): Die Anzahl der zu extrahierenden Zeichen von links. Standardwert ist 4. 
        Der Standardwert wurde gewählt, da die Funktion vorwiegend dafür da ist, 
        das Jahr aus einer Spalte im JJJJMM-Format zu extrahieren.

    Returns:
    str: Die ersten n-Zeichen des Eingabestrings.
    '''

    return val[:n]

In [16]:
def right(val, n = 2):
    '''
    Extrahiert die letzten n-Zeichen eines Strings.

    Args:
    val (str): Der Eingabestring.
    n (int): Die Anzahl der zu extrahierenden Zeichen von rechts. Standardwert ist 2. 
        Der Standardwert wurde gewählt, da die Funktion vorwiegend dafür da ist, 
        den Monat aus einer Spalte im JJJJMM-Format zu extrahieren.

    Returns:
    str: Die letzten n-Zeichen des Eingabestrings.
    '''

    return val[-n:]

In [22]:
data['monat_jahr'] = data['MONAT'].astype(str).apply(lambda x: left(x)).astype(int)
data['monat_monat'] = data['MONAT'].astype(str).apply(lambda x: right(x)).astype(int)

data['kohorte_jahr'] = data['KOHORTE'].astype(str).apply(lambda x: left(x)).astype(int)
data['kohorte_monat'] = data['KOHORTE'].astype(str).apply(lambda x: right(x)).astype(int)

In [27]:
# Überprüfung, ob Funktionen richtig funktioniert haben
data[['MONAT', 'monat_jahr', 'monat_monat', 'KOHORTE', 'kohorte_jahr', 'kohorte_monat']].head()

,MONAT,monat_jahr,monat_monat,KOHORTE,kohorte_jahr,kohorte_monat
0,201408,2014,8,201408,2014,8
1,201409,2014,9,201408,2014,8
2,201410,2014,10,201408,2014,8
3,201411,2014,11,201408,2014,8
4,201412,2014,12,201408,2014,8


Um die Jahreszeiten zu bestimmen wird eine einfache Aufteilung der Monate in Jahreszeiten angenommen (z.B. Dezemeber bis Februar entspricht Winter). Auch hierfür wird zunächst eine Funktion definiert.

In [28]:
def get_season(month):
    '''
    Ordnet den angegebenen Monat der entsprechenden Jahreszeit zu. 
    
    Diese Funktion nimmt einen Monat als Integer und gibt die Jahreszeit zurück, 
    zu der der Monat gehört. Die Einteilung erfolgt nach folgenden Kriterien: 
    - Dezember, Januar, Februar: Winter 
    - März, April, Mai: Frühling 
    - Juni, Juli, August: Sommer 
    - September, Oktober, November: Herbst 
    
    Args: 
    month (int): Der Monat als Integer (1 bis 12). 
    
    Returns: 
    str: Die Jahreszeit ('Winter', 'Frühling', 'Sommer', 'Herbst')
    '''
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Frühling'
    elif month in [6, 7, 8]:
        return 'Sommer'
    elif month in [9, 10, 11]:
        return 'Herbst'


In [29]:
data['monat_jahreszeit'] = data['monat_monat'].apply(lambda x: get_season(x))

data['kohorte_jahreszeit'] = data['kohorte_monat'].apply(lambda x: get_season(x))

In [30]:
# Überprüfung, ob Funktionen richtig funktioniert haben
data[['monat_monat', 'monat_jahreszeit', 'kohorte_monat', 'kohorte_jahreszeit']].head()

,monat_monat,monat_jahreszeit,kohorte_monat,kohorte_jahreszeit
0,8,Sommer,8,Sommer
1,9,Herbst,8,Sommer
2,10,Herbst,8,Sommer
3,11,Herbst,8,Sommer
4,12,Winter,8,Sommer


Die neu generieten Spalten stehen nun am Ende des Dataframes. Um dies übersichtlich zu gestalten, werden die Spalten entsprechend umgeordnet.

In [32]:
def rearrange_col(df, startpoint_column, rearrange_columns, inbetween=1):
    '''
    Strukturiert das DataFrame um, indem mehrere Spalten an neue Positionen verschoben werden.

    Diese Funktion nimmt ein DataFrame als Eingabe und verschiebt mehrere Spalten an neue Positionen,
    basierend auf einer angegebenen Startspalte und einem optionalen Abstand.

    Args:
    df (pd.DataFrame): Das DataFrame, das umstrukturiert werden soll.
    startpoint_column (str): Der Name der Spalte, deren Position als Ausgangspunkt genommen wird.
    rearrange_columns (list): Eine Liste der Spaltennamen, die umstrukturiert werden sollen.
    inbetween (int, optional): Die Anzahl der Spalten, die die umstrukturierten Spalten nach der Ausgangsspalte einnehmen sollen. Standard ist 1.

    Returns:
    pd.DataFrame: Das umstrukturierte DataFrame.
    '''
    
    # Bestimmt die Position der Ausgangsspalte
    starting_point = df.columns.get_loc(startpoint_column)
    
    # Erstellt eine Liste aller Spaltennamen
    columns = df.columns.tolist()
    
     # Entfernt die umstrukturierten Spalten aus der Liste
    for col in rearrange_columns:
        columns.remove(col)
    
    # Fügt die umstrukturierten Spalten an den neuen Positionen wieder ein
    for i, col in enumerate(rearrange_columns):
        columns.insert(starting_point + inbetween + i, col)
    
    # Gibt das umstrukturierte DataFrame zurück
    return df[columns]


In [36]:
data = rearrange_col(data, 'MONAT', ['monat_jahr', 'monat_monat', 'monat_jahreszeit'])
data = rearrange_col(data, 'KOHORTE', ['kohorte_jahr', 'kohorte_monat', 'kohorte_jahreszeit'])

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703 entries, 0 to 702
Data columns (total 27 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   MONATE_SEIT_EINFUEHRUNG_PROGRAMM_KOHORTE  703 non-null    int64  
 1   MONAT                                     703 non-null    int64  
 2   monat_jahr                                703 non-null    int64  
 3   monat_monat                               703 non-null    int64  
 4   monat_jahreszeit                          703 non-null    object 
 5   KOHORTE                                   703 non-null    int64  
 6   kohorte_jahr                              703 non-null    int64  
 7   kohorte_monat                             703 non-null    int64  
 8   kohorte_jahreszeit                        703 non-null    object 
 9   ERSTER_MONAT_KOHORTE_FG                   703 non-null    int64  
 10  MONATE_SEIT_EXISTENZ_KOHORTE          

#### Daten ablegen
Nun wird die neue Datengrundlage in den Ordner interim abgelegt

In [39]:
path_data_new = 'c:/Users/miche/HdM/Data Analytics with Statistics/Git/project/data/interim/'
file_data_new = 'Datengrundlage_added_features.csv'

data.to_csv(path_data_new + file_data_new, index = False)